## Evaluation of Models on Incident Prediction

In [4]:
import numpy as np
import torch

from torchmetrics import Accuracy
# from torchmetrics.functional import precision_recall
from tqdm import tqdm
from datetime import datetime as dt
from matplotlib import pyplot as plt

from train import create_parser
from models import *

### 1. Load Model and Data

In [5]:
parser = create_parser()
args = parser.parse_args(args=[])

# For reproducibility
seed_torch(args.seed)

In [6]:
args.use_density = True
args.use_truck_spd = True
args.use_pv_spd = True
args.use_speed = True
args.use_expectation = True
args.device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [7]:
args.county = "TSMO"
args.dim_hidden = 1024
args.LR_pos_weight = 0.2442
args.dim_in = 4099
args.dim_out = 583
args.batch_size = 256

In [8]:
traffic_model = TransFact(args).to(args.device)
traffic_model_path = "./checkpoints/TSMO/finetune/best_Trans_T_T_T_T_7_6_5_hidden_1024_batch_256_lr_0.00025_T.pt"
# base_model = Seq2SeqNoFact(args).to(args.device)
# base_model_path = "./checkpoints/base/best_exp_T_T_T_7_6_5_F.pt"
X_path = "../data/TSMO/TSMO_np_in_5min.npy"
Y_path = "../data/TSMO/TSMO_np_out_5min.npy"
inc_path = "../data/waze_out_5_min_v2.npy"
sigm = torch.nn.Sigmoid()

In [10]:
# with open(base_model_path, "rb") as f_base_model, open(traffic_model_path, "rb") as f_traffic_model:
with open(traffic_model_path, "rb") as f_traffic_model:
    # load state dict
    # state_dict_base = torch.load(f_base_model, map_location=args.device)
    state_dict_traffic = torch.load(f_traffic_model, map_location=args.device)["model"]

    # populate state dict into models and log
    # base_model.load_state_dict(state_dict_base)
    traffic_model.load_state_dict(state_dict_traffic)

In [21]:
input.shape

torch.Size([46800, 4099])

In [24]:
days = 260
daily_slots = 180

input = torch.from_numpy(np.load(X_path)).float()  # (46800, 4099)
output = torch.from_numpy(np.load(Y_path)).float()  # (46800, 583, 5)
# waze_out = torch.from_numpy(np.load(inc_path)).float()  # (21060, 75)

X = []  # serve as model input
Y = []  # serve as model input
target = []  # ground truth
inc_waze_gt = []

for d in tqdm(range(days)):
    for i in range(daily_slots-args.seq_len_in+1):
        # X.append(input[d*days+i : d*days+i+args.seq_len_in, :])
        # Y.append(output[d*days+i : d*days+i+args.seq_len_in, ...])
        X.append(input[d*daily_slots+i : d*daily_slots+i+args.seq_len_in, :])
        Y.append(output[d*daily_slots+i : d*daily_slots+i+args.seq_len_out+1, ...])
        target.append(output[d*daily_slots+i+1 : d*daily_slots+i+args.seq_len_out+1, ...])
X = torch.stack(X, dim=0).to(args.device)  # (45240, 7, 4099)
Y = torch.stack(Y, dim=0).to(args.device)  # (45240, 7, 583, 5)
target = torch.stack(target, dim=0).to(args.device)  # (45240, 6, 583, 5)
# inc_waze_gt = torch.stack(inc_waze_gt, dim=0).int().to(args.device)  # (20358, 6, 75)
# inc_target = target[...,1].int()  # (20358, 6, 75)
# spd_target = target[..., :1]  # (20358, 6, 75, 1)

100%|██████████| 260/260 [00:00<00:00, 353.91it/s]


### 2. Generate Model Inference

In [26]:
batch_size = 24
traffic_spd_pred, traffic_inc_pred, base_spd_pred = [], [], []
with torch.no_grad():
    traffic_model.eval()
    # base_model.eval()
    for i in tqdm(range(1885)):
        temp_X = X[(i*24) : (i+1)*24]
        temp_Y = Y[(i*24) : (i+1)*24]
        temp_traffic_spd_pred, temp_traffic_inc_pred, _ = traffic_model(temp_X, temp_Y, mode="eval")
        # temp_base_spd_pred, _, _ = base_model(temp_X, temp_Y)

        traffic_spd_pred.append(temp_traffic_spd_pred)
        traffic_inc_pred.append(temp_traffic_inc_pred)
        # base_spd_pred.append(temp_base_spd_pred)

traffic_spd_pred = torch.cat(traffic_spd_pred, dim=0)  # (45240, 6, 583)
traffic_inc_pred = sigm(torch.cat(traffic_inc_pred, dim=0))  # (45240, 6, 583)
# base_spd_pred = torch.cat(base_spd_pred, dim=0)  # (28536, 6, 210)

100%|██████████| 1885/1885 [03:24<00:00,  9.20it/s]


In [28]:
np.save("TSMO_traffic_spd_pred.npy", traffic_spd_pred.cpu().detach().numpy())
np.save("TSMO_traffic_inc_pred.npy", traffic_inc_pred.cpu().detach().numpy())
# np.save("TSMO_base_spd_pred.npy", base_spd_pred.cpu().detach().numpy())

### 3. Effect of Incident Prediction Threshold on Accuracy VS Recall

In [39]:
thresholds = [0.05*i for i in range(20)]
accu = []
recall = []
for t in thresholds:
    accu_metric = Accuracy(t)
    accu.append(accu_metric(sigm(traffic_inc_pred), inc_target))
    recall.append(precision_recall(sigm(traffic_inc_pred), inc_target, threshold=t)[1])

In [40]:
thres_accu_recall = [np.array(thresholds), torch.tensor(accu).numpy(), torch.tensor(recall).numpy()]
thres_accu_recall = np.stack(thres_accu_recall, axis=0)
np.save("thres_accu_recall", thres_accu_recall)

### 4. Case Study of Timeliness
- Here we use the actual event of Waze report: TMC 104-4441 (19th out segment, index 18) 17:15:00 on Mar 5, 2019 (24th date, index 23, starting from Feb 10, 2019)

- We check our model prediction on time slots 16:50:00~17:15:00 on Mar 5, 2019

In [98]:
time_idx = 663 # 3*174 + 141 #23*174 + 129
seg_idx = 18
selected_inc_pred = traffic_inc_pred[time_idx, :, seg_idx]
# selected_inc_target = inc_target[time_idx, :, seg_idx]
selected_inc_target = inc_waze_gt[time_idx, :, seg_idx]

selected_traffic_spd_pred = traffic_spd_pred.reshape(-1, args.seq_len_out, args.out_dim)[time_idx, :, seg_idx]
selected_spd_target = spd_target[time_idx, :, seg_idx]
selected_base_spd_pred = base_spd_pred.reshape(-1, args.seq_len_out, args.out_dim)[time_idx, :, seg_idx]

In [97]:
inc_waze_gt[663, :, seg_idx]

tensor([1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.int32)

In [99]:
selected_inc_pred, selected_inc_target

(tensor([0.8859, 0.8529, 0.8343, 0.8417, 0.8405, 0.8346], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.int32))

In [100]:
selected_traffic_spd_pred.T

tensor([58.9597, 65.3369, 66.4474, 67.2265, 66.0705, 71.2776], device='cuda:0')

In [101]:
selected_spd_target.T

tensor([[68., 68., 67., 62., 62., 72.]], device='cuda:0')

In [102]:
selected_base_spd_pred.T

tensor([65.0479, 60.7633, 62.5623, 62.9894, 61.3666, 60.7738], device='cuda:0')

In [123]:
# inference on 16:45:00, as well as the following time period (17:20:00 ~ 17:45:00), 
lag = -1 # 6
selected_traffic_spd_pred_2 = traffic_spd_pred.reshape(-1, args.seq_len_out, args.out_dim, 3)[time_idx+lag, :, seg_idx, :]
selected_spd_target_2 = spd_target[time_idx+lag, :, seg_idx, :]
selected_base_spd_pred_2 = base_spd_pred.reshape(-1, args.seq_len_out, args.out_dim, 3)[time_idx+lag, :, seg_idx, :]

In [124]:
traffic_inc_pred[time_idx-1, :, seg_idx]

tensor([0.7280, 0.7357, 0.7709, 0.7378, 0.7393, 0.7421])

In [125]:
selected_traffic_spd_pred_2.T

tensor([[66.8232, 66.8523, 65.1955, 63.1000, 64.8696, 65.2021],
        [64.6081, 64.8071, 64.0083, 62.9499, 63.9544, 64.0970],
        [68.7255, 68.4531, 67.1062, 66.4765, 67.6330, 67.7268]])

In [126]:
selected_spd_target_2.T

tensor([[68.0000, 63.0000, 59.0000, 64.0000, 62.0000, 63.0000],
        [64.6964, 59.9394, 60.0000, 60.0000, 64.0000, 56.0000],
        [68.0000, 63.0000, 59.0000, 65.0000, 61.0000, 65.0000]])

In [127]:
selected_base_spd_pred_2.T

tensor([[63.9186, 65.0571, 67.0426, 68.2490, 68.7815, 68.9281],
        [63.2506, 63.7744, 65.6257, 66.6814, 66.9713, 66.9547],
        [64.8901, 66.0237, 67.9922, 69.1591, 69.6962, 69.8517]])